# Load and save Landsat NDVI over Aus

`dea/20230710`

Need to install odc-geo for dea_tools to work

`pip install -v --no-binary :all: --upgrade-strategy only-if-needed --prefix /g/data/os22/chad_tmp/climate-carbon-interactions/envs/EXTRA_PYTHON_LIBS odc-geo`

### Load packages

In [ ]:
import datacube
import datacube.utils.rio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import folium

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.dask import create_local_dask_cluster
from dea_tools.bandindices import calculate_indices
from dea_tools.datahandling import load_ard

In [ ]:
client = create_local_dask_cluster(return_client=True)

## Analysis parameters


In [ ]:
measurements =  ['nbar_nir', 'nbar_red']
resampling = {'nbar_nir':'average', 'nbar_red':'average', '*':'mode'}
resolution = (-0.05, 0.05)
output_crs='epsg:4326'
dask_chunks=dict(latitude=1000, longitude=1000, time=1)
filters=[("opening", 2), ("dilation", 2)]

## Load data

In [ ]:
dc = datacube.Datacube(app='Landsat')

In [ ]:
#loop through years to keep memory down
years = [str(i) for i in range(2022,2023)]

for y in years:
    print(y)

    #set-up query
    query = {
        'time':y,
        'measurements': measurements,
        'resolution': resolution,
        'output_crs': output_crs,
        'resampling':resampling,
        'dask_chunks':dask_chunks,
        'group_by':'solar_day'
    }
    
    # with HiddenPrints():
    ds = load_ard(dc=dc,
              products=[
                    'ga_ls5t_ard_3',
                    'ga_ls7e_ard_3',
                    'ga_ls8c_ard_3',
                    'ga_ls9c_ard_3'
              ],
              mask_pixel_quality=True,
              mask_filters=filters,
              skip_broken_datasets=True,
              **query
             )

    #calculate NDVI and resample to monthly
    ds = calculate_indices(ds, ['NDVI'], collection='ga_ls_3', drop=True)
    ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).median()
    ds = ds.compute()
    #export
    del ds.attrs['grid_mapping']
    # for var in ds.data_vars:
    #     del ds[var].attrs['grid_mapping']
    ds.to_netcdf('/g/data/os22/chad_tmp/AusENDVI/data/Landsat/Landsat_'+y+'.nc')
    

In [ ]:
# del ds.attrs['grid_mapping']
# del ds.time.attrs['units']
# for var in ds.data_vars:
    # del ds[var].attrs['grid_mapping']

# ds.to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/Landsat/Landsat_'+y+'.nc')

## Stitch together output

In [ ]:
import os
import pandas as pd
import xarray as xr
import numpy as np
from odc.geo.xr import assign_crs
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _collect_prediction_data import round_coords

In [ ]:
base = '/g/data/os22/chad_tmp/climate-carbon-interactions/data/Landsat/'
files = [base+i for i in os.listdir(base) if i.endswith('.nc')]
files.sort()
print(len(files))

In [ ]:
arr = []
for f in files:
    ds = xr.open_dataset(f)
    ds = round_coords(ds)
    ds = assign_crs(ds, crs='epsg:4326')
    ds.attrs['nodata'] = np.nan
    arr.append(ds)
ds = xr.concat(arr, dim='time')
ds.attrs['nodata'] = np.nan

In [ ]:
gbox = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/WCF_5km_monthly_1982_2022.nc')['WCF'].odc.geobox
ds = ds.odc.reproject(gbox, resampling='bilinear')
ds = round_coords(ds)
ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).median()

In [ ]:
ds.NDVI.mean(['latitude', 'longitude']).plot(figsize=(11,4))

In [ ]:
ds.to_netcdf('/g/data/os22/chad_tmp/AusENDVI/data/Landsat_NDVI_5km_monthly_1988_2012.nc')